In [1]:
%matplotlib inline

import torch
import torchvision
import torch.nn.functional as F
import torch.optim as optim
import torch.nn as nn
from torch.autograd import Variable
import random as rm

# Add the sibling folders
import sys, os
sys.path.insert(0, os.path.abspath('../..'))
import src.utils as ut

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        # define all the components that will be used in the NN (these can be reused)
        self.fc1 = nn.Linear(784,100, bias=False)
        self.fc2 = nn.Linear(100,10, bias=False)
        
    def forward(self, x):
        # define the acutal network
        in_size = x.size(0) # get the batch size
        
        x = x.view(in_size, -1) # flatten data, -1 is inferred from the other dimensions
        
        x = F.relu(self.fc1(x))
        x = self.fc2(x) # Provare aggiungendo softmax
        return x

In [3]:
train_loader, test_loader = ut.load_dataset(dataset_name='mnist', minibatch=4096)

In [4]:
def test_minibatch(inputs, labels, model):
    model.eval()
    correct = 0
    total = 0
    test_loss = 0

    outputs = model(Variable(inputs))
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()
    test_loss += float(F.cross_entropy(outputs, Variable(labels)).item())

    return correct / total

In [5]:
def train(trainloader, model, optimizer, criterion, update_number, gpu=True):
    model.train()
    
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data

        if gpu:
            inputs, labels = inputs.cuda(), labels.cuda()
        print("Prima:", test_minibatch(inputs, labels, net))
        for j in range(update_number):
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            
            optimizer.step()
            
        if i % 10 and 0 == 1:
            '''for param in model.parameters():
                print(param.grad.mul(0.01), param.grad.size())'''
            print(test_minibatch(inputs, labels, net))
            
        print("Dopo:", test_minibatch(inputs, labels, net))
        del inputs, labels, outputs

## Mail
Una cosa che mi intriga è la seguente: noi sappiamo che, per f.o. L(w) differenziabili come quelli che stiamo considerando, esiste un gradiente 

$$ g(w) $$

che potremmo (ma non vogliamo) calcolare. Dato il set w dei parametri correnti, noi generiamo una “mossa” random \Delta(w) e valutiamo la f.o. L(w)  in 

$$ w' := w - \epsilon \Delta(w) $$

per un piccolo \epsilon > 0. Ora se la norma di \epsilon \Delta(w) è piccola noi sappiamo (approssimazione di Taylor) che 

$$ L(w’) ~= L(w) - \epsilon g(w)^T \Delta(w) $$

quindi la f.o. migliora se  

$$ g(w)^T \Delta(w) > 0 $$

(cioè se vettori -g(w) e \Delta(w) formano un angolo acuto). Se questo non succede, allora potremmo eseguire lo spostamento nella direzione contraria

$$	w" := w + \epsilon \Delta(w) $$ 

con la speranza di migliorare. 

In un’ottica Simulated Annealing, questo corrisponde a considerare come “mossa di base” (per un \Delta(w) random) il migliore dei due spostamenti

$$ w' := w - \epsilon \Delta(w) $$
$$ w" := w + \epsilon \Delta(w) $$

Se il migliore dei due migliora L(w) allora accettiamo certamente la mossa, se no la accettiamo con una probabilità legata alla temperatura, ecc

In [ ]:
maximum_percentage = 10e-1 / 2
net = Net().cuda()
train_loader, test_loader = ut.load_dataset(dataset_name='mnist', minibatch=4096)

def train_SA(trainloader, model, gpu=True):
    model.train()
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data

        if gpu:
            inputs, labels = inputs.cuda(), labels.cuda()
        
        if i == 0:
            accuracy_before = test_minibatch(inputs, labels, net)
        
        grad = None
        grad2 = None
        inverse = []
        
        for param in net.parameters():
            size = param.data.size()
            grad = param.data.clone()
            grad2 = param.data.clone()
            
            for i in range(size[0]):
                for j in range(size[1]):
                    grad.data[i][j] = rm.uniform(-1, 1) * maximum_percentage * param.data[i][j]
                    grad2.data[i][j] = grad.data[i][j] * 2 * -1
            #print(grad.data)
            inverse.append(grad2.data)
            param.data.add_(grad.data)
        
        new_accuracy = test_minibatch(inputs, labels, net)
        
        if new_accuracy < accuracy_before:
            print("Direzione sbagliata")
            for k, param in enumerate(net.parameters()):
                param.data.add_(inverse[k])
        
            new_accuracy = test_minibatch(inputs, labels, net)
            
        accuracy_before = new_accuracy
        
        print(new_accuracy)
        del inputs, labels
        
def full_train_SA(trainloader, model, gpu=True):
    model.train()
    
    if i
        accuracy_before = ut.test_train(trainloader, net)
    print("Accuracy inizio epoca:", accuracy_before)
    
    grad = None
    inverse = []
    
    for param in net.parameters():
        size = param.data.size()
        grad = param.data.clone()
        
        #print(param.data)
        for i in range(size[0]):
            for j in range(size[1]):
                grad.data[i][j] = rm.uniform(-1, 1) * maximum_percentage * param.data[i][j]
        
        inverse.append(grad.data.mul(-2))
        param.data.add_(grad.data)

    new_accuracy = ut.test_train(train_loader, net)

    if new_accuracy[1] < accuracy_before[1]:
        print("Direzione sbagliata infatti:", new_accuracy[1])
        for k, param in enumerate(net.parameters()):
            param.data.add_(inverse[k])

        new_accuracy = ut.test_train(train_loader, net)
        
    accuracy_before = new_accuracy

    print("Accuracy fine epoca: ", new_accuracy)
    
    
for epoch in range(1000):
    print("Epoch: ", epoch)
    full_train_SA(train_loader, net, 1)
    print("Validation test:", ut.test(test_loader, net))

Epoch:  0
Accuracy inizio epoca: (0.0005848147432009379, 0.13281666666666667)
Direzione sbagliata infatti: 0.12443333333333334
Accuracy fine epoca:  (0.0005837009827295939, 0.14145)
Epoch:  1
Accuracy inizio epoca: (0.0005837320526440938, 0.14145)
Direzione sbagliata infatti: 0.13806666666666667
Accuracy fine epoca:  (0.0005828081448872885, 0.14458333333333334)
Epoch:  2
Accuracy inizio epoca: (0.0005827558279037475, 0.14458333333333334)
Accuracy fine epoca:  (0.000582395871480306, 0.14865)
Epoch:  3
Accuracy inizio epoca: (0.0005823869387308756, 0.14865)
Direzione sbagliata infatti: 0.14725
Accuracy fine epoca:  (0.0005820811271667481, 0.14846666666666666)
Epoch:  4
Accuracy inizio epoca: (0.0005820894559224447, 0.14846666666666666)
Direzione sbagliata infatti: 0.13901666666666668
Accuracy fine epoca:  (0.0005819591800371805, 0.15708333333333332)
Epoch:  5
Accuracy inizio epoca: (0.0005819628755251567, 0.15708333333333332)
Direzione sbagliata infatti: 0.1545
Accuracy fine epoca:  (0.0

Direzione sbagliata infatti: 0.22775
Accuracy fine epoca:  (0.0005741359273592631, 0.22743333333333332)
Epoch:  50
Accuracy inizio epoca: (0.0005740385452906291, 0.22743333333333332)
Direzione sbagliata infatti: 0.22563333333333332
Accuracy fine epoca:  (0.0005733922799428304, 0.22838333333333333)
Epoch:  51
Accuracy inizio epoca: (0.0005734728495279948, 0.22838333333333333)
Accuracy fine epoca:  (0.000573296594619751, 0.2305)
Epoch:  52
Accuracy inizio epoca: (0.0005732896645863851, 0.2305)
Direzione sbagliata infatti: 0.22916666666666666
Accuracy fine epoca:  (0.0005732404788335165, 0.2299)
Epoch:  53
Accuracy inizio epoca: (0.000573265814781189, 0.2299)
Direzione sbagliata infatti: 0.22055
Accuracy fine epoca:  (0.0005717859586079915, 0.23791666666666667)
Epoch:  54
Accuracy inizio epoca: (0.0005717929085095724, 0.23791666666666667)
Direzione sbagliata infatti: 0.2342
Accuracy fine epoca:  (0.0005713620702425639, 0.24016666666666667)
Epoch:  55
Accuracy inizio epoca: (0.000571342233